In [1]:
import pandas as pd
import glob

In [2]:
directory = 'targeted'

In [3]:
count = 0

file_list = glob.glob(directory + "/*.xlsx")

excl_list = []
for file in file_list:
    excl_list.append(pd.read_excel(str(file), sheet_name = 'Tweets'))
    print(str(file))

excl_merged = pd.concat(excl_list, ignore_index=True)

targeted/targeted-10.xlsx
targeted/targeted-26.xlsx
targeted/targeted-7.xlsx
targeted/targeted-30.xlsx
targeted/targeted-31.xlsx
targeted/targeted-6.xlsx
targeted/targeted-27.xlsx
targeted/targeted-11.xlsx
targeted/targeted-20.xlsx
targeted/targeted-1.xlsx
targeted/targeted-36.xlsx
targeted/targeted-41.xlsx
targeted/targeted-16.xlsx
targeted/targeted-17.xlsx
targeted/targeted-40.xlsx
targeted/targeted-37.xlsx
targeted/targeted-21.xlsx
targeted/targeted-34.xlsx
targeted/targeted-3.xlsx
targeted/targeted-22.xlsx
targeted/targeted-18.xlsx
targeted/targeted-38.xlsx
targeted/targeted-14.xlsx
targeted/targeted-43.xlsx
targeted/targeted-42.xlsx
targeted/targeted-15.xlsx
targeted/targeted-39.xlsx
targeted/targeted-19.xlsx
targeted/targeted-23.xlsx
targeted/targeted-2.xlsx
targeted/targeted-35.xlsx
targeted/targeted-28.xlsx
targeted/targeted-9.xlsx
targeted/targeted-12.xlsx
targeted/targeted-45.xlsx
targeted/targeted-32.xlsx
targeted/targeted.xlsx
targeted/targeted-5.xlsx
targeted/targeted-24.x

In [6]:
pd.set_option('display.max_colwidth', None)
excl_merged.drop('Binders', axis = 1)


,User,Username,UTC Date,Tweet,Permalink,Retweet count,Likes count,Tweet value
0,Gary Hill,@hillg367,2023-01-30 19:39:18,"RT @davenewworld_2: 4 white supremacists in Washington state were sentenced to 28 to 48 months for a hate crime after they entered a bar and assaulted the DJ, Tyrone Smith, while calling him slurs. They targeted a Black man on ""Martyr's Day"" to ""honor"" a white supremacist who died in the 1980s. https://t.co/O3ojTWDpVL",https://www.twitter.com/user/status/1620144641940258816,0,0,0.46
1,خالد الشعيبي,@qwe716128968,2023-01-30 19:39:16,The US directly targeted all economic sectors in #Yemen and looted all the revenues leaving the state employees with nothing to live off of\n#USAEnemyOfPeace\n#امريكا_عدو_السلام,https://www.twitter.com/user/status/1620144636416380928,0,0,3.19
2,ampk.95,@ampk_95,2023-01-30 19:39:16,@mccnshyne @akitoshinonome Not really an otome at all. More just like you learn lore about the characters and listen to all the songs they got released. Theres a game where you can collect their cards. Technically targeted for women but like you just kinda hang out with them as they info dump their issues,https://www.twitter.com/user/status/1620144635808223233,0,0,0.01
3,عبدالله,@bdllh16716948,2023-01-30 19:39:15,Every aspect of life in #Yemen was targeted by the US-Saudi aggression even chicken farms and horse stables were not left untouched\n#USAEnemyOfPeace\n#امريكا_عدو_السلام\nhttps://t.co/ieboNZTpfD,https://www.twitter.com/user/status/1620144629135069184,0,0,0.06
4,llwry,@LLLwry,2023-01-30 19:39:14,"RT @davenewworld_2: 4 white supremacists in Washington state were sentenced to 28 to 48 months for a hate crime after they entered a bar and assaulted the DJ, Tyrone Smith, while calling him slurs. They targeted a Black man on ""Martyr's Day"" to ""honor"" a white supremacist who died in the 1980s. https://t.co/O3ojTWDpVL",https://www.twitter.com/user/status/1620144627256025088,0,0,5.10
...,...,...,...,...,...,...,...,...
459854,Jamie Chirps 🦜🇨🇦,@JamieLC6,2023-01-19 23:21:16,RT @Roman_Baber: ICYMI. The same Canadian Armed Forces team that engaged in propaganda against Canadians at the start of the Pandemic has targeted Canadians again. \n\n@AnitaAnandMP &amp; @JustinTrudeau are spreading propaganda and using the @CanadianForces against Canadians. https://t.co/doJZQXCdns,https://www.twitter.com/user/status/1616214235918356481,0,0,0.41
459855,UnacceptableSilverFox 🍁,@Cdn_Eliana,2023-01-19 23:21:08,RT @Roman_Baber: ICYMI. The same Canadian Armed Forces team that engaged in propaganda against Canadians at the start of the Pandemic has targeted Canadians again. \n\n@AnitaAnandMP &amp; @JustinTrudeau are spreading propaganda and using the @CanadianForces against Canadians. https://t.co/doJZQXCdns,https://www.twitter.com/user/status/1616214205010694145,0,0,5.38
459856,SteelerMoo,@FatherMoo,2023-01-19 23:21:06,"RT @ESPNRingside: Stephen Fulton and Naoya Inoue have agreed to terms for a unified 122-pound title fight in the spring in Japan, sources told @MikeCoppinger.\n\nThe fight for Fulton's WBC and WBO titles is being targeted for May, though no date is set yet. The bout will be streamed on @ESPNPlus. https://t.co/e0TtMsTGoP",https://www.twitter.com/user/status/1616214194184937474,0,0,1.22
459857,fiVth,@VthpoetV,2023-01-19 23:21:06,"RT @TasetiReloaded: After the expulsion of the Moors, the Christian church SANCTIONED the enslavement of Africans. Africans were a TARGETED population. Europeans went to Africa with PREMEDITATION to enslave. Inconsequential if some Africans were sellers b/c Euros would enslave Africans thru FORCE.",https://www.twitter.com/user/status/1616214192918249472,0,0,0.17


In [7]:
excl_merged['Tweet']

0           RT @davenewworld_2: 4 white supremacists in Washington state were sentenced to 28 to 48 months for a hate crime after they entered a bar and assaulted the DJ, Tyrone Smith, while calling him slurs. They targeted a Black man on "Martyr's Day" to "honor" a white supremacist who died in the 1980s. https://t.co/O3ojTWDpVL
1                                                                                                                                                         The US directly targeted all economic sectors in #Yemen and looted all the revenues leaving the state employees with nothing to live off of\n#USAEnemyOfPeace\n#امريكا_عدو_السلام
2                        @mccnshyne @akitoshinonome Not really an otome at all. More just like you learn lore about the characters and listen to all the songs they got released. Theres a game where you can collect their cards. Technically targeted for women but like you just kinda hang out with them as they info dump their issues
3   

In [ ]:
excl_merged.to_csv('excl_merged.csv')

In [ ]:
df = pd.read_csv('excl_merged.csv')
df

,Unnamed: 0,User,Username,UTC Date,Tweet,Binders,Permalink,Retweet count,Likes count,Tweet value
0,0,Gary Hill,@hillg367,2023-01-30 19:39:18,RT @davenewworld_2: 4 white supremacists in Wa...,NaN,https://www.twitter.com/user/status/1620144641...,0,0,0.46
1,1,خالد الشعيبي,@qwe716128968,2023-01-30 19:39:16,The US directly targeted all economic sectors ...,NaN,https://www.twitter.com/user/status/1620144636...,0,0,3.19
2,2,ampk.95,@ampk_95,2023-01-30 19:39:16,@mccnshyne @akitoshinonome Not really an otome...,NaN,https://www.twitter.com/user/status/1620144635...,0,0,0.01
3,3,عبدالله,@bdllh16716948,2023-01-30 19:39:15,Every aspect of life in #Yemen was targeted by...,NaN,https://www.twitter.com/user/status/1620144629...,0,0,0.06
4,4,llwry,@LLLwry,2023-01-30 19:39:14,RT @davenewworld_2: 4 white supremacists in Wa...,NaN,https://www.twitter.com/user/status/1620144627...,0,0,5.10
...,...,...,...,...,...,...,...,...,...,...
459854,459854,Jamie Chirps 🦜🇨🇦,@JamieLC6,2023-01-19 23:21:16,RT @Roman_Baber: ICYMI. The same Canadian Arme...,NaN,https://www.twitter.com/user/status/1616214235...,0,0,0.41
459855,459855,UnacceptableSilverFox 🍁,@Cdn_Eliana,2023-01-19 23:21:08,RT @Roman_Baber: ICYMI. The same Canadian Arme...,NaN,https://www.twitter.com/user/status/1616214205...,0,0,5.38
459856,459856,SteelerMoo,@FatherMoo,2023-01-19 23:21:06,RT @ESPNRingside: Stephen Fulton and Naoya Ino...,NaN,https://www.twitter.com/user/status/1616214194...,0,0,1.22
459857,459857,fiVth,@VthpoetV,2023-01-19 23:21:06,RT @TasetiReloaded: After the expulsion of the...,NaN,https://www.twitter.com/user/status/1616214192...,0,0,0.17


In [ ]:
df['Tweet']

0           RT @davenewworld_2: 4 white supremacists in Washington state were sentenced to 28 to 48 months for a hate crime after they entered a bar and assaulted the DJ, Tyrone Smith, while calling him slurs. They targeted a Black man on "Martyr's Day" to "honor" a white supremacist who died in the 1980s. https://t.co/O3ojTWDpVL
1                                                                                                                                                         The US directly targeted all economic sectors in #Yemen and looted all the revenues leaving the state employees with nothing to live off of\n#USAEnemyOfPeace\n#امريكا_عدو_السلام
2                        @mccnshyne @akitoshinonome Not really an otome at all. More just like you learn lore about the characters and listen to all the songs they got released. Theres a game where you can collect their cards. Technically targeted for women but like you just kinda hang out with them as they info dump their issues
3   